# CellDART Example Code: mouse brain 
## (10x Visium of anterior mouse brain + scRNA-seq data of mouse brain)

In [1]:
import glob
import os
import pickle
from math import ceil

import anndata as ad
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import scanpy as sc
import seaborn as sns
import scipy.stats as ss

from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import mean_squared_error
from sklearn import metrics

from CellDART import da_cellfraction
from CellDART.utils import random_mix

from src.utils.data_loading import load_spatial, load_sc, get_selected_dir

from tqdm.autonotebook import tqdm


/tmp/ipykernel_2920/1012129130.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import tensorflow as tf   # TensorFlow registers PluggableDevices here.
tf.config.list_physical_devices() 

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
TRAIN_USING_ALL_ST_SAMPLES = False
N_MARKERS = 20
ALL_GENES = False

ST_SPLIT = False
N_SPOTS = 20000
N_MIX = 8
SCALER_NAME = "celldart"


SAMPLE_ID_N = "151673"
INITIAL_TRAIN_EPOCHS = 10


BATCH_SIZE = 512
ALPHA = 0.6
ALPHA_LR = 5
N_ITER = 3000

DATA_DIR = "../AGrEDA/data"

BOOTSTRAP = False
BOOTSTRAP_ROUNDS = 10
BOOTSTRAP_ALPHAS = [0.6, 1/0.6]

MODEL_NAME = 'CellDART'
MODEL_VERSION = "V1"

In [4]:
model_folder = os.path.join("model", MODEL_NAME, MODEL_VERSION)

if not os.path.isdir(model_folder):
    os.makedirs(model_folder)
    print(model_folder)


In [5]:
def get_selected_dir(data_dir, n_markers=20, all_genes=False):
    if all_genes:
        return os.path.join(data_dir, "preprocessed", "all")
    else:
        return os.path.join(data_dir, "preprocessed", f"{n_markers}markers")

## 1. Data load
### load scanpy data - 10x datasets

In [6]:
# Load spatial data
mat_sp_d, mat_sp_train, st_sample_id_l = load_spatial(
    get_selected_dir(DATA_DIR, N_MARKERS, ALL_GENES),
    SCALER_NAME,
    train_using_all_st_samples=TRAIN_USING_ALL_ST_SAMPLES,
    st_split=ST_SPLIT,
)

# Load sc data
sc_mix_d, lab_mix_d, sc_sub_dict, sc_sub_dict2 = load_sc(
    get_selected_dir(DATA_DIR, N_MARKERS, ALL_GENES),
    SCALER_NAME,
    n_mix=N_MIX,
    n_spots=N_SPOTS,
)


In [7]:
advtrain_folder = os.path.join(model_folder, "advtrain")
pretrain_folder = os.path.join(model_folder, "pretrain")
if not os.path.isdir(advtrain_folder):
    os.makedirs(advtrain_folder)
if not os.path.isdir(pretrain_folder):
    os.makedirs(pretrain_folder)

In [9]:
if TRAIN_USING_ALL_ST_SAMPLES:
    print(f"Adversarial training for all ST slides")
    embs, clssmodel, clssmodel_noda = da_cellfraction.train(
        sc_mix_d["train"],
        lab_mix_d["train"],
        mat_sp_train_s,
        alpha=ALPHA,
        alpha_lr=5,
        emb_dim=64,
        batch_size=BATCH_SIZE,
        n_iterations=N_ITER,
        initial_train=True,
        initial_train_epochs=INITIAL_TRAIN_EPOCHS,
    )
elif BOOTSTRAP:
    pred_sp_boostrap_d = {}
    
    outer = tqdm(total=len(BOOTSTRAP_ALPHAS), desc="Alphas", position=0)
    inner1 = tqdm(total=len(st_sample_id_l), desc=f"Sample", position=1)
    inner2 = tqdm(total=BOOTSTRAP_ROUNDS, desc=f"Bootstrap #", position=2)
    for alpha in BOOTSTRAP_ALPHAS:
        
        inner1.refresh()  # force print final state
        inner1.reset()  # reuse bar

        
        pred_sp_boostrap_d[alpha] = {}
        

        for sample_id in st_sample_id_l:
            inner2.refresh()  # force print final state
            inner2.reset()  # reuse bar
            
            pred_sp_boostrap_d[alpha][sample_id] = []
            for i in range(BOOTSTRAP_ROUNDS):
                print(f"Adversarial training for ST slide {sample_id}: ")
                embs, clssmodel, _ = da_cellfraction.train(
                    sc_mix_d["train"],
                    lab_mix_d["train"],
                    mat_sp_d["train"][sample_id],
                    alpha=alpha,
                    alpha_lr=5,
                    emb_dim=64,
                    batch_size=BATCH_SIZE,
                    n_iterations=N_ITER,
                    initial_train=True,
                    initial_train_epochs=10,
                    seed = i,
                )

                pred_sp_boostrap_d[alpha][sample_id].append(clssmodel.predict(
                     mat_sp_d["train"][sample_id]
                ))
                inner2.update(1)
            inner1.update(1)
        outer.update(1)

else:
    # embs_d, clssmodel_d, clssmodel_noda_d = {}, {}, {}
    for sample_id in st_sample_id_l:
        print(f"Adversarial training for ST slide {sample_id}: ")
        if not os.path.isdir(os.path.join(advtrain_folder, sample_id)):
            os.makedirs(os.path.join(advtrain_folder, sample_id))
        if not os.path.isdir(os.path.join(pretrain_folder, sample_id)):
            os.makedirs(os.path.join(pretrain_folder, sample_id))
        embs, embs_noda, clssmodel, clssmodel_noda = da_cellfraction.train(
            sc_mix_d["train"],
            lab_mix_d["train"],
            mat_sp_d[sample_id]["train"],
            alpha=ALPHA,
            alpha_lr=5,
            emb_dim=64,
            batch_size=BATCH_SIZE,
            n_iterations=N_ITER,
            initial_train=True,
            initial_train_epochs=10,
        )
        # embs_d[sample_id] = embs
        # clssmodel_d[sample_id] = clssmodel
        # clssmodel_noda_d[sample_id] = clssmodel_noda

        # Save model
        clssmodel_noda.save(os.path.join(pretrain_folder, sample_id, "final_model"))
        clssmodel.save(os.path.join(advtrain_folder, sample_id, "final_model"))

        embs_noda.save(os.path.join(pretrain_folder, sample_id, "embs"))
        embs.save(os.path.join(advtrain_folder, sample_id, "embs"))


Adversarial training for ST slide 151507: 
Train on 20000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 105us/sample - loss: 1.1898 - mae: 0.0343
Epoch 2/10
20000/20000 [==============================] - 0s 10us/sample - loss: 0.8301 - mae: 0.0281
Epoch 3/10
20000/20000 [==============================] - 0s 10us/sample - loss: 0.6762 - mae: 0.0249
Epoch 4/10
20000/20000 [==============================] - 0s 11us/sample - loss: 0.6071 - mae: 0.0230
Epoch 5/10
20000/20000 [==============================] - 0s 10us/sample - loss: 0.5751 - mae: 0.0220
Epoch 6/10
20000/20000 [==============================] - 0s 11us/sample - loss: 0.5534 - mae: 0.0214
Epoch 7/10
20000/20000 [==============================] - 0s 10us/sample - loss: 0.5371 - mae: 0.0210
Epoch 8/10
20000/20000 [==============================] - 0s 11us/sample - loss: 0.5224 - mae: 0.0206
Epoch 9/10
20000/20000 [==============================] - 0s 11us/sample - loss: 0.5116 - mae: 0.0203
Epoch 10/10
200

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Iteration 99, source loss =  4.918, discriminator acc = 0.174
Iteration 199, source loss =  2.263, discriminator acc = 0.463
Iteration 299, source loss =  1.516, discriminator acc = 0.173
Iteration 399, source loss =  1.667, discriminator acc = 0.174
Iteration 499, source loss =  1.583, discriminator acc = 0.174
Iteration 599, source loss =  1.393, discriminator acc = 0.174
Iteration 699, source loss =  1.592, discriminator acc = 0.234
Iteration 799, source loss =  1.433, discriminator acc = 0.828
Iteration 899, source loss =  1.264, discriminator acc = 0.244
Iteration 999, source loss =  1.579, discriminator acc = 0.936
Iteration 1099, source loss =  1.166, discriminator acc = 0.404
Iteration 1199, source loss =  1.632, discriminator acc = 0.852
Iteration 1299, source loss =  1.121, discriminator acc = 0.254
Iteration 1399, source loss =  1.172, discriminator acc = 0.790
Iteration 1499, source loss =  1.006, discriminator acc = 0.813
Iteration 1599, source loss =  0.947, discriminator

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Iteration 99, source loss =  3.589, discriminator acc = 0.180
Iteration 199, source loss =  2.009, discriminator acc = 0.232
Iteration 299, source loss =  1.777, discriminator acc = 0.887
Iteration 399, source loss =  1.220, discriminator acc = 0.948
Iteration 499, source loss =  1.608, discriminator acc = 0.732
Iteration 599, source loss =  1.313, discriminator acc = 0.855
Iteration 699, source loss =  1.117, discriminator acc = 0.964
Iteration 799, source loss =  1.088, discriminator acc = 0.997
Iteration 899, source loss =  0.959, discriminator acc = 0.714
Iteration 999, source loss =  0.956, discriminator acc = 0.117
Iteration 1099, source loss =  0.788, discriminator acc = 0.931
Iteration 1199, source loss =  0.786, discriminator acc = 0.984
Iteration 1299, source loss =  0.786, discriminator acc = 0.055
Iteration 1399, source loss =  0.760, discriminator acc = 0.376
Iteration 1499, source loss =  0.801, discriminator acc = 0.106
Iteration 1599, source loss =  0.860, discriminator

...

Iteration 99, source loss =  1.965, discriminator acc = 0.155
Iteration 199, source loss =  2.012, discriminator acc = 0.865
Iteration 299, source loss =  1.902, discriminator acc = 0.865
Iteration 399, source loss =  1.483, discriminator acc = 0.852
Iteration 499, source loss =  1.588, discriminator acc = 0.865
Iteration 599, source loss =  1.532, discriminator acc = 0.873
Iteration 699, source loss =  1.283, discriminator acc = 0.874
Iteration 799, source loss =  1.294, discriminator acc = 0.878
Iteration 899, source loss =  1.209, discriminator acc = 0.846
Iteration 999, source loss =  1.249, discriminator acc = 0.866
Iteration 1099, source loss =  1.163, discriminator acc = 0.878
Iteration 1199, source loss =  1.253, discriminator acc = 0.954
Iteration 1299, source loss =  1.286, discriminator acc = 0.863
Iteration 1399, source loss =  1.358, discriminator acc = 0.851
Iteration 1499, source loss =  1.245, discriminator acc = 0.868
Iteration 1599, source loss =  1.233, discriminator

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Iteration 99, source loss =  2.303, discriminator acc = 0.865
Iteration 199, source loss =  2.572, discriminator acc = 0.863
Iteration 299, source loss =  1.606, discriminator acc = 0.860
Iteration 399, source loss =  1.874, discriminator acc = 0.871
Iteration 499, source loss =  1.294, discriminator acc = 0.882
Iteration 599, source loss =  1.523, discriminator acc = 0.873
Iteration 699, source loss =  1.451, discriminator acc = 0.866
Iteration 799, source loss =  1.544, discriminator acc = 0.884
Iteration 899, source loss =  1.544, discriminator acc = 0.884
Iteration 999, source loss =  1.328, discriminator acc = 0.881
Iteration 1099, source loss =  1.350, discriminator acc = 0.887
Iteration 1199, source loss =  1.423, discriminator acc = 0.880
Iteration 1299, source loss =  1.338, discriminator acc = 0.884
Iteration 1399, source loss =  1.201, discriminator acc = 0.888
Iteration 1499, source loss =  1.299, discriminator acc = 0.877
Iteration 1599, source loss =  1.260, discriminator

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Iteration 99, source loss =  4.264, discriminator acc = 0.072
Iteration 199, source loss =  1.424, discriminator acc = 0.100
Iteration 299, source loss =  1.700, discriminator acc = 0.160
Iteration 399, source loss =  1.227, discriminator acc = 0.975
Iteration 499, source loss =  1.461, discriminator acc = 0.937
Iteration 599, source loss =  1.639, discriminator acc = 0.136
Iteration 699, source loss =  1.269, discriminator acc = 1.000
Iteration 799, source loss =  1.082, discriminator acc = 0.627
Iteration 899, source loss =  1.180, discriminator acc = 0.943
Iteration 999, source loss =  1.098, discriminator acc = 0.148
Iteration 1099, source loss =  1.017, discriminator acc = 0.953
Iteration 1199, source loss =  1.008, discriminator acc = 0.643
Iteration 1299, source loss =  0.909, discriminator acc = 0.896
Iteration 1399, source loss =  1.012, discriminator acc = 0.791
Iteration 1499, source loss =  0.958, discriminator acc = 0.362
Iteration 1599, source loss =  0.907, discriminator

In [ ]:
# confidence = 0.95
# if BOOTSTRAP:
#     for alpha in BOOTSTRAP_ALPHAS:
#         bootstrap_d = {}
#         for sample_id in st_sample_id_l:
            
            
#             bootstrap_d[sample_id] = {}
#             for i, num in enumerate(numlist):
                
#                 acc = [metrics.roc_auc_score(*gen_pred_true(num, adata_spatialLIBD_d[SAMPLE_ID_N], pred)[::-1]) for pred in pred_sp_boostrap_d[alpha][SAMPLE_ID_N]]
#                 test_mean = np.mean(acc)
#                 t_value = ss.t.ppf((1 + confidence) / 2.0, df=BOOTSTRAP_ROUNDS - 1)

#                 sd = np.std(acc, ddof=1)
#                 se = sd / np.sqrt(BOOTSTRAP_ROUNDS)

#                 ci_length = t_value * se

#                 ci_lower = test_mean - ci_length
#                 ci_upper = test_mean + ci_length

#                 bootstrap_d[sample_id][num_to_ex_d[num]] = (ci_lower, test_mean, ci_upper)


#         bootstrap_df = pd.DataFrame.from_dict(bootstrap_d)
#         display(bootstrap_df)
#         bootstrap_df.to_csv(os.path.join(results_folder, f'bootstrap_alpha{alpha}.csv'))

## 4. Predict cell fraction of spots and visualization

In [ ]:
# pred_sp_d, pred_sp_noda_d = {}, {}
# if TRAIN_USING_ALL_ST_SAMPLES:
#     for sample_id in st_sample_id_l:
#         pred_sp_d[sample_id] = clssmodel.predict(mat_sp_test_s_d[sample_id])
#         pred_sp_noda_d[sample_id] = clssmodel_noda.predict(mat_sp_test_s_d[sample_id])
# else:
#     for sample_id in st_sample_id_l:
#         pred_sp_d[sample_id] = clssmodel_d[sample_id].predict(
#             mat_sp_test_s_d[sample_id]
#         )
#         pred_sp_noda_d[sample_id] = clssmodel_noda_d[sample_id].predict(
#             mat_sp_test_s_d[sample_id]
#         )



In [ ]:
# def plot_cellfraction(visnum, adata, pred_sp, ax=None):
#     """Plot predicted cell fraction for a given visnum"""
#     adata.obs["Pred_label"] = pred_sp[:, visnum]
#     # vmin = 0
#     # vmax = np.amax(pred_sp)

#     sc.pl.spatial(
#         adata,
#         img_key="hires",
#         color="Pred_label",
#         palette="Set1",
#         size=1.5,
#         legend_loc=None,
#         title=f"{sc_sub_dict[visnum]}",
#         spot_size=100,
#         show=False,
#         # vmin=vmin,
#         # vmax=vmax,
#         ax=ax,
#     )


In [ ]:
# def plot_cell_layers(df):

#     layer_idx = df["spatialLIBD"].unique().astype(str)
#     samples = df["sample_id"].unique()
#     layer_idx.sort()
#     fig, ax = plt.subplots(
#         nrows=1,
#         ncols=len(samples),
#         figsize=(5 * len(samples), 5),
#         squeeze=False,
#         constrained_layout=True,
#     )

#     for idx, sample in enumerate(samples):
#         cells_of_samples = df[df["sample_id"] == sample]
#         for index in layer_idx:
#             cells_of_layer = cells_of_samples[cells_of_samples["spatialLIBD"] == index]
#             ax.flat[idx].scatter(
#                 cells_of_layer["X"], -cells_of_layer["Y"], label=index, s=17, marker="o"
#             )

#         ax.flat[idx].axis("equal")
#         ax.flat[idx].set_xticks([])
#         ax.flat[idx].set_yticks([])
#         ax.flat[idx].set_title(sample)

#     plt.legend()
#     plt.show()


In [ ]:
# def plot_roc(visnum, adata, pred_sp, name, ax=None):
#     """Plot ROC for a given visnum"""

#     def layer_to_layer_number(x):
#         for char in x:
#             if char.isdigit():
#                 if int(char) in Ex_to_L_d[num_to_ex_d[visnum]]:
#                     return 1
#         return 0

#     y_pred = pred_sp[:, visnum]
#     y_true = adata.obs["spatialLIBD"].map(layer_to_layer_number).fillna(0)
#     # print(y_true)
#     # print(y_true.isna().sum())
#     RocCurveDisplay.from_predictions(y_true=y_true, y_pred=y_pred, name=name, ax=ax)


In [ ]:
# # plot_cell_layers(adata_spatialLIBD_151673.obs)

# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5), constrained_layout=True)

# sc.pl.spatial(
#     adata_spatialLIBD_d[SAMPLE_ID_N],
#     img_key=None,
#     color="spatialLIBD",
#     palette="Accent_r",
#     size=1.5,
#     title=SAMPLE_ID_N,
#     # legend_loc = 4,
#     spot_size=100,
#     show=False,
#     ax=ax,
# )

# ax.axis("equal")
# ax.set_xlabel("")
# ax.set_ylabel("")

# fig.show()


In [ ]:
# fig, ax = plt.subplots(2, 5, figsize=(20, 8), constrained_layout=True)

# for i, num in enumerate(numlist):
#     plot_cellfraction(
#         num, adata_spatialLIBD_d[SAMPLE_ID_N], pred_sp_d[SAMPLE_ID_N], ax.flat[i]
#     )
#     ax.flat[i].axis("equal")
#     ax.flat[i].set_xlabel("")
#     ax.flat[i].set_ylabel("")

# fig.show()
# # plt.close()

# fig, ax = plt.subplots(
#     2, 5, figsize=(20, 8), constrained_layout=True, sharex=True, sharey=True
# )

# for i, num in enumerate(numlist):
#     plot_roc(
#         num,
#         adata_spatialLIBD_d[SAMPLE_ID_N],
#         pred_sp_d[SAMPLE_ID_N],
#         "CellDART",
#         ax.flat[i],
#     )
#     plot_roc(
#         num,
#         adata_spatialLIBD_d[SAMPLE_ID_N],
#         pred_sp_noda_d[SAMPLE_ID_N],
#         "NN_wo_da",
#         ax.flat[i],
#     )
#     ax.flat[i].plot([0, 1], [0, 1], transform=ax.flat[i].transAxes, ls="--", color="k")
#     ax.flat[i].set_aspect("equal")
#     ax.flat[i].set_xlim([0, 1])
#     ax.flat[i].set_ylim([0, 1])

#     ax.flat[i].set_title(f"{sc_sub_dict[num]}")

#     if i >= len(numlist) - 5:
#         ax.flat[i].set_xlabel("FPR")
#     else:
#         ax.flat[i].set_xlabel("")
#     if i % 5 == 0:
#         ax.flat[i].set_ylabel("TPR")
#     else:
#         ax.flat[i].set_ylabel("")

# fig.show()
# # plt.close()


- cf. Prediction of Mixture (pseudospots)


In [ ]:
# if TRAIN_USING_ALL_ST_SAMPLES:
#     pred_mix = clssmodel.predict(sc_mix_test_s)
# else:
#     pred_mix = clssmodel_d[SAMPLE_ID_N].predict(sc_mix_test_s)


# cell_type_nums = sc_sub_dict.keys()
# nrows = ceil(len(cell_type_nums) / 5)

# line_kws = {"color": "tab:orange"}
# scatter_kws = {"s": 5}

# props = dict(facecolor="w", alpha=0.5)

# fig, ax = plt.subplots(
#     nrows,
#     5,
#     figsize=(20, 4 * nrows),
#     constrained_layout=True,
#     sharex=False,
#     sharey=True,
# )
# for i, num in enumerate(cell_type_nums):
#     sns.regplot(
#         x=pred_mix[:, num],
#         y=lab_mix_test[:, num],
#         line_kws=line_kws,
#         scatter_kws=scatter_kws,
#         ax=ax.flat[i],
#     ).set_title(sc_sub_dict[num])
#     ax.flat[i].set_aspect("equal")

#     ax.flat[i].set_xlabel("Predicted Proportion")
#     if i % 5 == 0:
#         ax.flat[i].set_ylabel("True Proportion")
#     else:
#         ax.flat[i].set_ylabel("")
#     ax.flat[i].set_xlim([0, 1])
#     ax.flat[i].set_ylim([0, 1])

#     textstr = f"MSE: {mean_squared_error(pred_mix[:,num], lab_mix_test[:,num]):.5f}"

#     # place a text box in upper left in axes coords
#     ax.flat[i].text(
#         0.95,
#         0.05,
#         textstr,
#         transform=ax.flat[i].transAxes,
#         verticalalignment="bottom",
#         horizontalalignment="right",
#         bbox=props,
#     )

# for i in range(len(cell_type_nums), nrows * 5):
#     ax.flat[i].axis("off")

# plt.show()


In [ ]:
# print(
#     "\n".join(
#         f"{m.__name__} {m.__version__}"
#         for m in globals().values()
#         if getattr(m, "__version__", None)
#     )
# )
